In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [2]:
spark=SparkSession.builder.appName("Recommender").getOrCreate()

24/11/05 19:58:06 WARN Utils: Your hostname, Anants-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.86.5.30 instead (on interface en0)
24/11/05 19:58:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/05 19:58:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1) Demonstrate how to load a dataset suitable for recommendation systems into a PySpark DataFrame.

In [4]:
rating_df=spark.read.options(delimiter=";", header=True, inferSchema=True).csv("ratings.csv")
rating_df.show()

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
| 276733|2080674722|          0|
| 276736|3257224281|          8|
| 276737|0600570967|          6|
| 276744|038550120X|          7|
| 276745| 342310538|         10|
| 276746|0425115801|          0|
| 276746|0449006522|          0|
| 276746|0553561618|          0|
| 276746|055356451X|          0|
| 276746|0786013990|          0|
| 276746|0786014512|          0|
| 276747|0060517794|          9|
| 276747|0451192001|          0|
| 276747|0609801279|          0|
| 276747|0671537458|          9|
+-------+----------+-----------+
only showing top 20 rows



In [5]:
rating_df.describe().show()

+-------+------------------+-----------+------------------+
|summary|           User-ID|       ISBN|       Book-Rating|
+-------+------------------+-----------+------------------+
|  count|           1149780|    1149780|           1149780|
|   mean|140386.39512602412|   Infinity|2.8669501991685364|
| stddev| 80562.27771851176|        NaN| 3.854183859201656|
|    min|                 2| 0330299891|                 0|
|    max|            278854| �423350229|                10|
+-------+------------------+-----------+------------------+



2) Implement a PySpark script that splits the data and trains a recommendation model.

Preprocessing data to be fed into ALS

In [8]:
from pyspark.ml.feature import StringIndexer

user_Indexer=StringIndexer(inputCol="User-ID",outputCol="User-ID-numeric")
item_Indexer=StringIndexer(inputCol="ISBN",outputCol="ISBN-numeric")

user_indexer_model = user_Indexer.fit(rating_df)
item_indexer_model = item_Indexer.fit(rating_df)

rating_df=user_indexer_model.transform(rating_df)

rating_df=item_indexer_model.transform(rating_df)


In [9]:
train_data,test_data=rating_df.randomSplit([0.8,0.2])


3) Implement a PySpark script using the ALS algorithm for collaborative filtering.

In [11]:
train_data.show(5)

24/11/05 19:58:16 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB


+-------+----------+-----------+---------------+------------+
|User-ID|      ISBN|Book-Rating|User-ID-numeric|ISBN-numeric|
+-------+----------+-----------+---------------+------------+
|      2|0195153448|          0|        69494.0|    160911.0|
|      7| 034542252|          0|        98245.0|     93830.0|
|      8|0002005018|          5|         7987.0|     10774.0|
|      8|0060973129|          0|         7987.0|     58687.0|
|      8|0374157065|          0|         7987.0|     14866.0|
+-------+----------+-----------+---------------+------------+
only showing top 5 rows



class pyspark.ml.recommendation.ALS(
rank=10,
maxIter=10,
regParam=0.1,
numUserBlocks=10,
numItemBlocks=10,
implicitPrefs=False,
alpha=1.0,
userCol="user",
itemCol="item",
seed=None,
ratingCol="rating",
nonnegative=False,
checkpointInterval=10,
intermediateStorageLevel="MEMORY_AND_DISK",
finalStorageLevel="MEMORY_AND_DISK",
coldStartStrategy="nan",
)

In [14]:
als=ALS(maxIter=10,regParam=0.1, userCol="User-ID-numeric",itemCol="ISBN-numeric",ratingCol="Book-Rating",coldStartStrategy="drop", nonnegative=True)
model=als.fit(train_data)

24/11/05 19:58:36 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:37 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:39 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:42 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:45 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:46 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:48 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/11/05 19:58:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/11/05 19:58:49 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:58:51 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB


4) Implement code to evaluate the performance of the recommendation model using appropriate metrics.

In [16]:
predictions=model.transform(test_data)
predictions.show()

24/11/05 19:59:22 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:24 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:26 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:29 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/11/05 19:59:31 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB


+-------+----------+-----------+---------------+------------+----------+
|User-ID|      ISBN|Book-Rating|User-ID-numeric|ISBN-numeric|prediction|
+-------+----------+-----------+---------------+------------+----------+
|  26374|0446605239|          5|        18896.0|        26.0| 3.8595567|
| 278543|0446605239|          5|        21939.0|        26.0|       0.0|
|  36606|0446605239|         10|           43.0|        26.0|0.82669485|
|  24186|0446605239|          0|         2278.0|        26.0|  4.480584|
|   6772|0446605239|          5|         2490.0|        26.0|  3.222149|
|   3460|0446605239|          6|        42910.0|        26.0|       0.0|
|  33319|0446605239|          0|         1767.0|        26.0|0.95971084|
|  31040|0446605239|          0|         1472.0|        26.0|0.50545466|
|  16718|0446605239|          9|         2302.0|        26.0|  8.603321|
|  11676|0446605239|          6|            0.0|        26.0| 2.2041748|
|  30898|0446605239|          4|         1929.0|   

Evaluation

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(metricName="rmse",labelCol="Book-Rating",predictionCol="prediction")
rmse=evaluator.evaluate(predictions)
print(rmse)

24/11/05 19:59:31 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:32 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:33 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:37 WARN DAGScheduler: Broadcasting large task binary with size 4.3 MiB
24/11/05 19:59:38 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB
24/11/05 19:59:40 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB


4.021936046385591


In [19]:
test_data.show()

24/11/05 19:59:41 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB


+-------+----------+-----------+---------------+------------+
|User-ID|      ISBN|Book-Rating|User-ID-numeric|ISBN-numeric|
+-------+----------+-----------+---------------+------------+
|      8|074322678X|          5|         7987.0|    234221.0|
|      8|1552041778|          5|         7987.0|    278428.0|
|      9|0452264464|          6|        33336.0|       247.0|
|     16|0345402871|          9|        36526.0|       183.0|
|     17|0553278398|          0|        16147.0|      1895.0|
|     17|0684823802|          0|        16147.0|     17725.0|
|     22|3404921038|          7|        24796.0|    135518.0|
|     32|038078243X|          0|        31875.0|     19140.0|
|     39|0553582909|          8|        32046.0|      4984.0|
|     44|0842342702|          0|        22143.0|       488.0|
|     69|0617683993|          6|        19443.0|    217139.0|
|     75|0345465083|          0|        26620.0|       225.0|
|     75|1558531025|          5|        26620.0|       898.0|
|     85

In [20]:
user1=test_data.filter(test_data['user-ID']==53).select(['User-ID-numeric','ISBN-numeric','Book-Rating'])
user1.show()

24/11/05 19:59:41 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:42 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:43 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB


+---------------+------------+-----------+
|User-ID-numeric|ISBN-numeric|Book-Rating|
+---------------+------------+-----------+
+---------------+------------+-----------+



In [21]:
recommendations = model.transform(user1) 

recommendations.orderBy('prediction',ascending=False).show()

24/11/05 19:59:44 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:45 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB


+---------------+------------+-----------+----------+
|User-ID-numeric|ISBN-numeric|Book-Rating|prediction|
+---------------+------------+-----------+----------+
+---------------+------------+-----------+----------+



24/11/05 19:59:46 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB


In [22]:
from pyspark.sql.functions import col
books_df=spark.read.options(delimiter=";", header=True, inferSchema=True).csv("books.csv")

def recommend_for_user(user_id, n):
    ratings_user = rating_df.filter(col('User-Id')==user_id)
    #getting books where rating is still zero or zero interaction
    pred_ratings_user = model.transform(ratings_user.filter(col('Book-Rating')==0))
    recs_user = books_df.join(pred_ratings_user.select(['ISBN', 'prediction']), on='ISBN')
    recs_user = recs_user.sort('prediction', ascending=False).limit(10)
    # recs_user = recs_user.sort('prediction', ascending=False).drop('prediction').limit(10)
    return recs_user
recs_user = recommend_for_user(31987, 7)
recs_user.show()

24/11/05 19:59:51 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:52 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:53 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 19:59:57 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB
24/11/05 19:59:58 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB


+----------+--------------------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      ISBN|          Book-Title|     Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|prediction|
+----------+--------------------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+
|0671690094|ANNE FRANK: DIARY...|      Anne Frank|               1989|              Pocket|http://images.ama...|http://images.ama...|http://images.ama...|  3.214597|
|0316666343|The Lovely Bones:...|    Alice Sebold|               2002|       Little, Brown|http://images.ama...|http://images.ama...|http://images.ama...| 2.5179114|
|0446603589|      Absolute Power|  David Baldacci|               1996|        Warner Books|http://images.ama...|http://images.ama...|http://images.ama...| 1.9852843|
|067

In [23]:
from pyspark.ml.tuning import CrossValidator,  ParamGridBuilder 
parameter_grid = (
    ParamGridBuilder()
    .addGrid(als.rank, [1, 5])
    .addGrid(als.maxIter, [20])
    .addGrid(als.regParam, [0.05])
    .addGrid(als.alpha,[1])
    .build()
)
print(type(parameter_grid))
from pprint import pprint
pprint(parameter_grid)
cross_validator=CrossValidator(
    estimator=als,
    estimatorParamMaps=parameter_grid,
    evaluator=evaluator,
    numFolds=2,
)
crossval_model=cross_validator.fit(train_data)
predictions=crossval_model.transform(test_data)
rmse=evaluator.evaluate(predictions.na.drop())
print(rmse)
predictions.show()
model=crossval_model.bestModel
print(model)

<class 'list'>
[{Param(parent='ALS_51167c3da79a', name='alpha', doc='alpha for implicit preference'): 1.0,
  Param(parent='ALS_51167c3da79a', name='maxIter', doc='max number of iterations (>= 0).'): 20,
  Param(parent='ALS_51167c3da79a', name='regParam', doc='regularization parameter (>= 0).'): 0.05,
  Param(parent='ALS_51167c3da79a', name='rank', doc='rank of the factorization'): 1},
 {Param(parent='ALS_51167c3da79a', name='alpha', doc='alpha for implicit preference'): 1.0,
  Param(parent='ALS_51167c3da79a', name='maxIter', doc='max number of iterations (>= 0).'): 20,
  Param(parent='ALS_51167c3da79a', name='regParam', doc='regularization parameter (>= 0).'): 0.05,
  Param(parent='ALS_51167c3da79a', name='rank', doc='rank of the factorization'): 5}]


24/11/05 20:00:00 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:03 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:03 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:05 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:06 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:08 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:09 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:11 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:13 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:14 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:15 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:00:17 WARN DAGScheduler: Broadc

4.261020967084558


24/11/05 20:06:10 WARN DAGScheduler: Broadcasting large task binary with size 13.7 MiB
24/11/05 20:06:11 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB
24/11/05 20:06:12 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB
24/11/05 20:06:15 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB


+-------+----------+-----------+---------------+------------+----------+
|User-ID|      ISBN|Book-Rating|User-ID-numeric|ISBN-numeric|prediction|
+-------+----------+-----------+---------------+------------+----------+
|  26374|0446605239|          5|        18896.0|        26.0|  8.179842|
| 278543|0446605239|          5|        21939.0|        26.0|       0.0|
|  36606|0446605239|         10|           43.0|        26.0|  1.373503|
|  24186|0446605239|          0|         2278.0|        26.0| 6.3180075|
|   6772|0446605239|          5|         2490.0|        26.0| 3.5577195|
|   3460|0446605239|          6|        42910.0|        26.0|       0.0|
|  33319|0446605239|          0|         1767.0|        26.0|  3.526819|
|  31040|0446605239|          0|         1472.0|        26.0| 1.3223614|
|  16718|0446605239|          9|         2302.0|        26.0|  4.323647|
|  11676|0446605239|          6|            0.0|        26.0| 4.6647806|
|  30898|0446605239|          4|         1929.0|   

24/11/05 20:06:16 WARN DAGScheduler: Broadcasting large task binary with size 13.8 MiB


In [21]:
spark.stop()

NameError: name 'spark' is not defined